Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "KIRILL GROMOV"
COLLABORATORS = ""

---

# I. $LU$ - разложение квадратной матрицы



Рассмотрим наивную реализацию LU - разложения.

Заметим, что мы используем массивы numpy для представления матриц. [Не используйте 'np.matrix'].


In [2]:
import numpy as np

def diy_lu(a):
    """Создает LU - разложение матрицы `a`.
    
    Наивное LU - разложение: работает столбец за столбцом, накапливает элементарные треугольные матрицы.
    Без выбора главного элемента.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [3]:
# Теперь сгенерируем матрицу полного ранга и протестируем наивное разложение.
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L,U = diy_lu(a)

np.linalg.matrix_rank(a)

6

In [4]:
# Настройка вывода чисел с плавающей точкой для большей ясности
np.set_printoptions(precision=3)

In [5]:
L, u = diy_lu(a)
print(L, "\n")
print(u, "\n")

# Быстрый тест на адекватность: L @ U должна быть равна изначальной матрице с точностью до ошибок округления.
print(a-L@u)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.220e-16  1.110e-16  1.665e-16]
 [ 0.000e+00  0.000e+00 -2.220e-16  5.551e-17  1.665e-16  1.665e-16]
 [ 0.000e+00  0.000e+00  1.110e-16  1.665e-16 -1.665e-16 -5.551e-17]
 

# II. Необходимость выбора главного элемента

Давайте немного подправим матрицу, изменив в ней один элемент:

In [6]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Результирующая матрица имеет полный ранг, но наивное LU-разложение не работает.

In [7]:
np.linalg.matrix_rank(a1)

6

In [8]:
l, u = diy_lu(a1)

print(l, u, sep='\n')

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/home/kotya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Тест II.1

Для того, чтобы наивное LU - разложение работало необходимо чтобы все лидирующие миноры матрицы были отличны от нуля. Проверьте, выполнено ли это требование для двух матриц a и a1.

In [9]:
def minor(a):
    ''' Check if all leading minors are non-zero.
    
    Parameters
    ----------
    a : np.array
        2D array representing the square matrix
    
    Returns
    -------
    answer : bool
        True if all leading minors are non-zero
    '''
    # YOUR CODE HERE
    import math
    n = len(a)
    minors = []
    result = True
    for k in range(1, n+1):
      minor_matrix = np.zeros((k, k))
      for i in range(k):
        for j in range(k):
          minor_matrix[i][j] = a[i][j]
      leading_minor = np.linalg.det(minor_matrix)
      # minors.append(leading_minor)
      if math.isclose(leading_minor, 0, abs_tol=1e-10):
        result = False
    return result
    # raise NotImplementedError()

In [10]:
assert minor(a)==True
assert minor(a1)==False

### Тест II.2

Модифицируйте алгоритм diy_lu, чтобы осуществлять выбор главного элемента в столбцах. Для контроля выбора можете использовать матрицу перестановок или массив замен.
Напишите функцию, воссоздающую изначальную матрицу из разложения. Протестируйте свой алгоритм на матрицах a и a1.

In [11]:
def diy_lu_mod(a):
    '''Perform pivoted LU factorization of the input matrix. 
    
    Parameters
    ----------
    a : np.array
        2D array representing a square matrix with float entries
       
    Returns
    -------
    P, L, U : ndarrays
        factors. Here P is a permutation matrix, L is lower triangular
        with unit diagonal elements, and U upper triangular.
    '''
    # YOUR CODE HERE
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
      max = 0.
      maxnum = j
      for k in range(j, N):
        if abs(u[k][j]) > max:
          max = u[k][j]
          maxnum = k
      u[j], u[maxnum] = u[maxnum], u[j].copy()
      P[j], P[maxnum] = P[maxnum], P[j].copy()
      lam = np.eye(N)
      gamma = u[j+1:, j] / u[j, j]
      lam[j+1:, j] = -gamma
      u = lam @ u

      lam[j+1:, j] = gamma
      L = L @ lam
    return np.linalg.inv(P), L, u
    # raise NotImplementedError()

from scipy import linalg
P1, L1, U1 = linalg.lu(a1)
P, L, U = diy_lu_mod(a1)

In [12]:
P, L, U = diy_lu_mod(a)
P1, L1, U1 = diy_lu_mod(a1)

from numpy.testing import assert_allclose
assert_allclose(a, P @ L @ U, atol=1e-10)
assert_allclose(a1, P1 @ L1 @ U1, atol=1e-10 )

###BEGIN HIDDEN TESTS
np.random.seed(1234)
m = np.random.rand(N, N)
a2 = np.diag([100]*N) + m.T @ m
P2, L2, U2 = diy_lu_mod(a2)

assert_allclose(a2, P2 @ L2 @ U2)
###END HIDDEN TESTS

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-10

Mismatched elements: 20 / 36 (55.6%)
Max absolute difference: 0.993
Max relative difference: 5.769
 x: array([[3.      , 3.      , 3.      , 3.      , 3.      , 3.      ],
       [3.      , 1.875   , 1.363636, 1.071429, 0.882353, 0.75    ],
       [3.      , 1.363636, 0.882353, 0.652174, 0.517241, 0.428571],...
 y: array([[3.      , 3.      , 3.      , 3.      , 3.      , 3.      ],
       [3.      , 1.363636, 0.779221, 0.457792, 0.252982, 0.110795],
       [3.      , 1.071429, 0.652174, 0.47275 , 0.374538, 0.313015],...